In [1]:
import tensorflow as tf
# from tensorflow.keras import layers
from tensorflow import keras
from keras import layers
from sklearn import  pipeline, preprocessing
from torchvision.datasets import EMNIST

from torch.utils.data import DataLoader

2024-01-29 14:35:17.768247: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 14:35:17.768386: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 14:35:18.046520: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 14:35:18.987014: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 14:35:22.892814: W tensorflow/compiler/tf2

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-01-29 14:35:38.339140: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-29 14:35:38.402002: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-29 14:35:38.402371: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:


# Load the datasets
train_data = EMNIST(root='emnist_data/', split='byclass', train=True, download=True)
test_data = EMNIST(root='emnist_data/', split='byclass', train=False, download=True)

# train_data_loader = DataLoader(train_data, batch_size=64, shuffle=True)
# test_data_loader = DataLoader(test_data, batch_size=64, shuffle=True)

# train_images = tf.data.Dataset.from_generator(lambda: train_data_loader, output_signature=tf.TensorSpec(shape=(None, 28, 28), dtype=tf.float32))
# test_images = tf.data.Dataset.from_generator(lambda: test_data_loader, output_signature=tf.TensorSpec(shape=(None, 28, 28), dtype=tf.float32))
#unpack the data into images and labels
train_images, train_labels = train_data.data, train_data.targets
test_images, test_labels = test_data.data, test_data.targets

#convert the data to numpy arrays
train_images = train_images.numpy()
train_labels = train_labels.numpy()
test_images = test_images.numpy()
test_labels = test_labels.numpy()


In [4]:
data_preprocessing_pipeline = pipeline.make_pipeline(
    preprocessing.FunctionTransformer(lambda X: tf.convert_to_tensor(X))
)

train_images = data_preprocessing_pipeline.fit_transform(train_images)
train_labels = data_preprocessing_pipeline.fit_transform(train_labels)

test_images = data_preprocessing_pipeline.fit_transform(test_images)
test_labels = data_preprocessing_pipeline.fit_transform(test_labels)

2024-01-29 14:35:48.768108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-29 14:35:48.768682: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-29 14:35:48.769043: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
testing_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [6]:


model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(1024, activation="tanh"),
    layers.Dense(1024//2, activation="tanh"),
    layers.Dense(1024//4, activation="tanh"),
    layers.Dense(62, activation="softmax")
])

In [7]:

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [8]:
# the training loop
epochs = 10
batch_size = 64

model.fit(training_dataset.shuffle(10000).batch(batch_size), epochs=epochs, batch_size=batch_size, validation_data=testing_dataset.batch(batch_size))

Epoch 1/10


2024-01-29 14:36:00.372313: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f5644203260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-29 14:36:00.372360: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 6GB, Compute Capability 6.1
2024-01-29 14:36:00.625361: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-29 14:36:01.026282: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1706528162.167427    3276 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10906/10906 [==============================] - 71s 6ms/step - loss: 1.1076 - accuracy: 0.6662 - val_loss: 0.9569 - val_accuracy: 0.7065
Epoch 2/10
 7055/10906 [==================>...........] - ETA: 21s - loss: 0.9765 - accuracy: 0.6982

KeyboardInterrupt: 

2024-01-28 19:18:14.475319: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 364788928 exceeds 10% of free system memory.
2024-01-28 19:18:14.707096: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 364788928 exceeds 10% of free system memory.


3636/3636 - 7s - loss: 0.7670 - accuracy: 0.7498 - 7s/epoch - 2ms/step
